#### Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** I used the same architecture as recommended in the documentation that was used in the "Show and Tell" paper: In the encoder part, resnet 50 is used followed by linear layer and batch normalization. In the decoder part, a single layer LSTM is used followed by linear layer. For selecting the parameters, I consulted "Show and Tell" (ST) & "Show, Attend, and Tell" (SAT) paper. As in ST paper, embedding size of 512 is used, and in SAT paper, batch size of 64 is used. For vocabulary, threhsold of 5 is used as was used in the ST paper for SBU data description and Karapathy's "Deep Visual Alignment" paper.


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** I sticked to the provided default transform values for the captioning task since it already had the crop operation (224 dimension) and normalization operation as required for the pre-trained ResNet. The random horizontal flipping operation helps with the logical augmentation that can be used in the training process.  

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** Linear Embedding layer in the CNN and all the parameters of decoder are trained. Since, we are using pre-trained ResNet there is no requirement to retrain them.

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** For optimization used the go-to optimizer algorithm of vision community - Adam. Adam combines the benefit of both RMSprop and Adagrad, and has been widely adopted in the community to tackle different problems.

In [3]:
%load_ext autoreload
%autoreload 2

!pip install nltk
import nltk
nltk.download('punkt')
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 64          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 1             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity
attention_dim = 512
alpha_c = 1.

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...




  0%|          | 0/414113 [00:00<?, ?it/s]

  0%|          | 337/414113 [00:00<02:02, 3367.68it/s]

  0%|          | 759/414113 [00:00<01:55, 3583.83it/s]

  0%|          | 1203/414113 [00:00<01:48, 3802.18it/s]

  0%|          | 1654/414113 [00:00<01:43, 3988.37it/s]

  1%|          | 2092/414113 [00:00<01:40, 4098.04it/s]

  1%|          | 2528/414113 [00:00<01:38, 4171.03it/s]

  1%|          | 2968/414113 [00:00<01:37, 4236.11it/s]

  1%|          | 3401/414113 [00:00<01:36, 4262.55it/s]

  1%|          | 3849/414113 [00:00<01:34, 4325.48it/s]

  1%|          | 4295/414113 [00:01<01:33, 4362.75it/s]

  1%|          | 4738/414113 [00:01<01:33, 4380.20it/s]

  1%|▏         | 5197/414113 [00:01<01:32, 4440.94it/s]

  1%|▏         | 5638/414113 [00:01<01:32, 4428.21it/s]

  1%|▏         | 6084/414113 [00:01<01:31, 4435.10it/s]

  2%|▏         | 6526/414113 [00:01<01:36, 4236.91it/s]

  2%|▏         | 6979/414113 [00:01<01:34, 4318.59it/s]

  2%|▏         | 7426/414113 [00:01<01:33, 4

 14%|█▍        | 57065/414113 [00:13<02:24, 2474.87it/s]

 14%|█▍        | 57517/414113 [00:13<02:04, 2862.63it/s]

 14%|█▍        | 57967/414113 [00:13<01:50, 3212.66it/s]

 14%|█▍        | 58419/414113 [00:13<01:41, 3516.29it/s]

 14%|█▍        | 58869/414113 [00:13<01:34, 3762.78it/s]

 14%|█▍        | 59309/414113 [00:13<01:30, 3931.33it/s]

 14%|█▍        | 59750/414113 [00:13<01:27, 4063.62it/s]

 15%|█▍        | 60199/414113 [00:13<01:24, 4181.55it/s]

 15%|█▍        | 60647/414113 [00:14<01:22, 4265.42it/s]

 15%|█▍        | 61090/414113 [00:14<01:21, 4311.83it/s]

 15%|█▍        | 61552/414113 [00:14<01:20, 4398.95it/s]

 15%|█▍        | 62006/414113 [00:14<01:19, 4439.16it/s]

 15%|█▌        | 62459/414113 [00:14<01:18, 4463.68it/s]

 15%|█▌        | 62919/414113 [00:14<01:18, 4501.59it/s]

 15%|█▌        | 63372/414113 [00:14<01:24, 4158.91it/s]

 15%|█▌        | 63823/414113 [00:14<01:22, 4256.09it/s]

 16%|█▌        | 64270/414113 [00:14<01:21, 4315.49it/s]

 16%|█▌       

 28%|██▊       | 113942/414113 [00:26<01:07, 4433.24it/s]

 28%|██▊       | 114394/414113 [00:26<01:07, 4457.98it/s]

 28%|██▊       | 114846/414113 [00:26<01:06, 4476.30it/s]

 28%|██▊       | 115294/414113 [00:26<01:07, 4452.85it/s]

 28%|██▊       | 115747/414113 [00:26<01:06, 4475.00it/s]

 28%|██▊       | 116195/414113 [00:26<01:06, 4469.76it/s]

 28%|██▊       | 116652/414113 [00:26<01:06, 4498.68it/s]

 28%|██▊       | 117102/414113 [00:26<01:06, 4479.17it/s]

 28%|██▊       | 117550/414113 [00:26<01:06, 4478.32it/s]

 28%|██▊       | 118000/414113 [00:26<01:06, 4483.27it/s]

 29%|██▊       | 118449/414113 [00:27<01:08, 4347.72it/s]

 29%|██▊       | 118906/414113 [00:27<01:06, 4409.87it/s]

 29%|██▉       | 119350/414113 [00:27<01:06, 4417.60it/s]

 29%|██▉       | 119799/414113 [00:27<01:06, 4436.84it/s]

 29%|██▉       | 120244/414113 [00:27<01:06, 4392.54it/s]

 29%|██▉       | 120684/414113 [00:27<01:06, 4389.29it/s]

 29%|██▉       | 121131/414113 [00:27<01:06, 4412.66it/s

 41%|████▏     | 170941/414113 [00:38<00:54, 4423.09it/s]

 41%|████▏     | 171391/414113 [00:38<00:54, 4443.42it/s]

 41%|████▏     | 171836/414113 [00:38<00:54, 4435.54it/s]

 42%|████▏     | 172285/414113 [00:39<00:54, 4449.91it/s]

 42%|████▏     | 172731/414113 [00:39<00:54, 4451.33it/s]

 42%|████▏     | 173177/414113 [00:39<00:54, 4449.69it/s]

 42%|████▏     | 173622/414113 [00:39<00:54, 4427.16it/s]

 42%|████▏     | 174065/414113 [00:39<01:43, 2311.37it/s]

 42%|████▏     | 174514/414113 [00:39<01:28, 2704.40it/s]

 42%|████▏     | 174970/414113 [00:40<01:17, 3079.44it/s]

 42%|████▏     | 175407/414113 [00:40<01:10, 3378.63it/s]

 42%|████▏     | 175858/414113 [00:40<01:05, 3652.28it/s]

 43%|████▎     | 176298/414113 [00:40<01:01, 3846.95it/s]

 43%|████▎     | 176733/414113 [00:40<00:59, 3982.86it/s]

 43%|████▎     | 177172/414113 [00:40<00:57, 4096.64it/s]

 43%|████▎     | 177613/414113 [00:40<00:56, 4184.94it/s]

 43%|████▎     | 178066/414113 [00:40<00:55, 4281.47it/s

 55%|█████▍    | 227747/414113 [00:51<00:40, 4602.00it/s]

 55%|█████▌    | 228208/414113 [00:51<00:41, 4439.96it/s]

 55%|█████▌    | 228666/414113 [00:52<00:41, 4480.19it/s]

 55%|█████▌    | 229123/414113 [00:52<00:41, 4506.64it/s]

 55%|█████▌    | 229575/414113 [00:52<00:40, 4509.08it/s]

 56%|█████▌    | 230027/414113 [00:52<00:41, 4478.18it/s]

 56%|█████▌    | 230476/414113 [00:52<00:41, 4408.34it/s]

 56%|█████▌    | 230921/414113 [00:52<00:41, 4418.73it/s]

 56%|█████▌    | 231364/414113 [00:52<00:42, 4335.27it/s]

 56%|█████▌    | 231809/414113 [00:52<00:41, 4368.32it/s]

 56%|█████▌    | 232247/414113 [00:52<00:42, 4303.43it/s]

 56%|█████▌    | 232682/414113 [00:52<00:42, 4316.97it/s]

 56%|█████▋    | 233122/414113 [00:53<00:41, 4339.79it/s]

 56%|█████▋    | 233575/414113 [00:53<00:41, 4394.91it/s]

 57%|█████▋    | 234018/414113 [00:53<00:40, 4404.55it/s]

 57%|█████▋    | 234476/414113 [00:53<00:40, 4454.94it/s]

 57%|█████▋    | 234924/414113 [00:53<00:40, 4462.21it/s

 69%|██████▊   | 284111/414113 [01:04<00:29, 4423.45it/s]

 69%|██████▊   | 284563/414113 [01:04<00:29, 4449.47it/s]

 69%|██████▉   | 285009/414113 [01:04<00:28, 4451.95it/s]

 69%|██████▉   | 285456/414113 [01:04<00:28, 4454.86it/s]

 69%|██████▉   | 285908/414113 [01:04<00:28, 4471.02it/s]

 69%|██████▉   | 286356/414113 [01:05<00:29, 4362.20it/s]

 69%|██████▉   | 286808/414113 [01:05<00:28, 4408.19it/s]

 69%|██████▉   | 287264/414113 [01:05<00:28, 4451.50it/s]

 69%|██████▉   | 287723/414113 [01:05<00:28, 4492.10it/s]

 70%|██████▉   | 288173/414113 [01:05<00:28, 4467.28it/s]

 70%|██████▉   | 288621/414113 [01:05<00:28, 4444.66it/s]

 70%|██████▉   | 289066/414113 [01:05<00:28, 4419.50it/s]

 70%|██████▉   | 289515/414113 [01:05<00:28, 4439.01it/s]

 70%|███████   | 289960/414113 [01:05<00:28, 4424.20it/s]

 70%|███████   | 290403/414113 [01:05<00:28, 4406.02it/s]

 70%|███████   | 290844/414113 [01:06<00:27, 4406.77it/s]

 70%|███████   | 291301/414113 [01:06<00:27, 4454.13it/s

 82%|████████▏ | 340127/414113 [01:17<00:17, 4351.54it/s]

 82%|████████▏ | 340567/414113 [01:17<00:16, 4363.54it/s]

 82%|████████▏ | 341008/414113 [01:17<00:16, 4376.02it/s]

 82%|████████▏ | 341446/414113 [01:17<00:16, 4365.95it/s]

 83%|████████▎ | 341883/414113 [01:18<00:16, 4309.22it/s]

 83%|████████▎ | 342316/414113 [01:18<00:16, 4312.68it/s]

 83%|████████▎ | 342754/414113 [01:18<00:16, 4331.04it/s]

 83%|████████▎ | 343197/414113 [01:18<00:16, 4357.84it/s]

 83%|████████▎ | 343645/414113 [01:18<00:16, 4392.12it/s]

 83%|████████▎ | 344093/414113 [01:18<00:15, 4417.38it/s]

 83%|████████▎ | 344535/414113 [01:18<00:15, 4410.52it/s]

 83%|████████▎ | 344986/414113 [01:18<00:15, 4437.73it/s]

 83%|████████▎ | 345430/414113 [01:18<00:15, 4416.03it/s]

 84%|████████▎ | 345880/414113 [01:18<00:15, 4439.91it/s]

 84%|████████▎ | 346327/414113 [01:19<00:15, 4446.76it/s]

 84%|████████▎ | 346772/414113 [01:19<00:15, 4438.80it/s]

 84%|████████▍ | 347216/414113 [01:19<00:15, 4413.82it/s

 96%|█████████▌| 395840/414113 [01:30<00:04, 4341.72it/s]

 96%|█████████▌| 396280/414113 [01:30<00:04, 4356.98it/s]

 96%|█████████▌| 396716/414113 [01:30<00:04, 4345.84it/s]

 96%|█████████▌| 397170/414113 [01:30<00:03, 4401.15it/s]

 96%|█████████▌| 397611/414113 [01:30<00:03, 4403.30it/s]

 96%|█████████▌| 398052/414113 [01:30<00:03, 4381.11it/s]

 96%|█████████▌| 398503/414113 [01:31<00:03, 4417.37it/s]

 96%|█████████▋| 398945/414113 [01:31<00:03, 4406.82it/s]

 96%|█████████▋| 399386/414113 [01:31<00:03, 4375.99it/s]

 97%|█████████▋| 399824/414113 [01:31<00:03, 4364.79it/s]

 97%|█████████▋| 400262/414113 [01:31<00:03, 4369.09it/s]

 97%|█████████▋| 400699/414113 [01:31<00:03, 4365.83it/s]

 97%|█████████▋| 401148/414113 [01:31<00:02, 4401.32it/s]

 97%|█████████▋| 401589/414113 [01:31<00:02, 4403.33it/s]

 97%|█████████▋| 402037/414113 [01:31<00:02, 4423.18it/s]

 97%|█████████▋| 402480/414113 [01:31<00:02, 4380.73it/s]

 97%|█████████▋| 402919/414113 [01:32<00:02, 4382.89it/s

Done (t=0.88s)
creating index...
index created!
Obtaining caption lengths...


In [5]:
from model_with_attention import *
# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size, attention_dim)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters())

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr = 0.001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [6]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        #encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs, alphas = decoder(features, captions)
        target = captions[:, 1:]
            
        # Include double stochastic regularization
        att_regularization = alpha_c * ((1 - alphas.sum(1))**2).mean()

        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), target.contiguous().view(-1))
        loss += att_regularization
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder_attention-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder_attention-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/1], Step [100/6471], Loss: 3.2921, Perplexity: 26.8990
Epoch [1/1], Step [200/6471], Loss: 3.0004, Perplexity: 20.0944
Epoch [1/1], Step [300/6471], Loss: 3.5867, Perplexity: 36.1153
Epoch [1/1], Step [400/6471], Loss: 3.1072, Perplexity: 22.3577
Epoch [1/1], Step [500/6471], Loss: 2.9599, Perplexity: 19.2958
Epoch [1/1], Step [600/6471], Loss: 2.9261, Perplexity: 18.6553
Epoch [1/1], Step [700/6471], Loss: 2.9366, Perplexity: 18.8525
Epoch [1/1], Step [800/6471], Loss: 3.3410, Perplexity: 28.2464
Epoch [1/1], Step [900/6471], Loss: 3.2334, Perplexity: 25.3646
Epoch [1/1], Step [1000/6471], Loss: 2.9464, Perplexity: 19.0379
Epoch [1/1], Step [1100/6471], Loss: 3.0481, Perplexity: 21.0747
Epoch [1/1], Step [1200/6471], Loss: 3.5220, Perplexity: 33.8516
Epoch [1/1], Step [1300/6471], Loss: 3.1706, Perplexity: 23.8215
Epoch [1/1], Step [1400/6471], Loss: 3.0854, Perplexity: 21.8762
Epoch [1/1], Step [1500/6471], Loss: 3.0687, Perplexity: 21.5146
Epoch [1/1], Step [1600/6471], Los

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.